In [1]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from urllib.parse import urlparse
from langdetect import detect
from textblob import TextBlob
#RegEx
import regex as re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1618926584724_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#Query some reviews
query_1 = "select distinct r.ugc_id, r.review_headline, r.review_comments, substr(r.locale,1,2) as lang, review_length, \
case when contains(r.observations, 'FL')  then 1 else 0 end as FL \
from analytics.review as r \
join analytics.merchant_group as mg \
on r.client_id = mg.client_id \
where contains(r.observations, 'PM') \
and not contains(r.observations, 'IA') and not contains(r.observations, 'IR') \
and mg.usage_type != 'Test' \
and r.created_date > '2021-01-01' \
and review_length <= 100 \
"

#193267606, 324912854
#and r.created_date > '2020-01-01' \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
reviews = pd.read_sql(query_1, conn)
reviews.head()
reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710235 entries, 0 to 1710234
Data columns (total 6 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   UGC_ID           int64 
 1   REVIEW_HEADLINE  object
 2   REVIEW_COMMENTS  object
 3   LANG             object
 4   REVIEW_LENGTH    int64 
 5   FL               int64 
dtypes: int64(3), object(3)
memory usage: 78.3+ MB

In [14]:
#Need to add "word" in front bc otherwise reviews with just emojis, symbols, #s, etc will not work in langdetect package
#Throws this error if no words: "langdetect.lang_detect_exception.LangDetectException: No features in text."
reviews['ALL_TEXT'] = reviews['REVIEW_HEADLINE'] + ' ' + reviews['REVIEW_COMMENTS']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)
#text = 'This is an english text.'
#doc = nlp(text)
#print(doc._.language)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
text = 'good good'
doc = nlp(text)
print(doc._.language)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'language': 'so', 'score': 0.9999971536415062}

In [9]:
def doc_lang(text):
    doc = nlp(text)
    return doc._.language

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
reviews['ALL_TEXT']=reviews['ALL_TEXT'].apply(str)
reviews['SPACY_LANG'] = reviews['ALL_TEXT'].apply(doc_lang)
reviews['PRED_LANG'] = [d.get('language') for d in reviews['SPACY_LANG']]
reviews['PROB'] = [d.get('score') for d in reviews['SPACY_LANG']]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

reviews.head(25)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       UGC_ID                                              REVIEW_HEADLINE  \
0   325766472  good bike                                                     
1   325153912  Motion Sickness on this machine                               
2   325242843  its a good bike                                               
3   330372758  It is a very nice i crashed into a car but the bike is ok     
4   340220163  This bike is made of 24k gold as much as a decent car         
5   330020372  wegweg                                                        
6   330946689  Just had my bike stolen and it was locked up with this lock   
7   338972858  yes                                                           
8   341353488  raquel                                                        
9   330658274  Defo one of the best bikes I've ever used except              
10  340032764  No                                                            
11  330485371  LOVE IT                                          

In [17]:
reviews['model_FL'] = 0

reviews.loc[reviews['LANG'] != reviews['PRED_LANG'] , 'model_FL'] = 1

#Our setup may have accidentally used "jp" as well as "ja" for Japenese
#"ja" is official code but mark "jp" as ok too
reviews.loc[ (reviews['LANG']=='jp') & (reviews['PRED_LANG']=='ja') , 'model_FL'] = 0



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
FP = reviews[(reviews['FL'] == 0) & (reviews['model_FL'] == 1)]
FN = reviews[(reviews['FL'] == 1) & (reviews['model_FL'] == 0)]
TP = reviews[(reviews['FL'] == 1) & (reviews['model_FL'] == 1)]
TN = reviews[(reviews['FL'] == 0) & (reviews['model_FL'] == 0)]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:

FP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

143340

In [20]:

FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

298

In [21]:

TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4509

In [22]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1562088

In [23]:
FP.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         UGC_ID                                        REVIEW_HEADLINE  \
5     330020372  wegweg                                                  
8     341353488  raquel                                                  
10    340032764  No                                                      
12    326463227  poopoo                                                  
20    324396691  Very Unique Gfit                                        
24    336539028  Hanger issues                                           
36    322736496  Great product                                           
62    329051808  weagwegaw                                               
328   322634553  Great puzzle detail - Alabama                           
380   332574928  Fun gift                                                
406   341210070  No idea                                                 
424   326843664  Beautiful crib board                                    
552   340230820  Flooring             

In [19]:
detect('good good stuff')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'cy'

In [24]:
FP.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143340 entries, 5 to 1710216
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   UGC_ID           143340 non-null  int64  
 1   REVIEW_HEADLINE  143329 non-null  object 
 2   REVIEW_COMMENTS  143340 non-null  object 
 3   LANG             143340 non-null  object 
 4   REVIEW_LENGTH    143340 non-null  int64  
 5   FL               143340 non-null  int64  
 6   ALL_TEXT         143340 non-null  object 
 7   SPACY_LANG       143340 non-null  object 
 8   PRED_LANG        143340 non-null  object 
 9   PROB             143340 non-null  float64
 10  model_FL         143340 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 13.1+ MB

In [25]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("LANG", StringType(), True)\
,StructField("REVIEW_LENGTH", StringType(), True)\
,StructField("FL", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("SPACY_LANG", StringType(), True)\
,StructField("PRED_LANG", StringType(), True)\
,StructField("PROB", FloatType(), True)\
,StructField("model_FL", IntegerType(), True)])




VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : username,
  'sfPassword' : password,
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
FN = spark.createDataFrame(FN, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
FN.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","FL_FN_SPACYLANGDETECT").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FP_TEST3")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FN_TEST3")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SPARK6")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SWAP_TOKENS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…